In [1]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time as t

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale

import tensorflow as tf
np.set_printoptions(precision=8)

seed = 0

In [3]:
# Read the file
file = 'D:/Education/HRR/molar_fraction_major_chemical_marker.hdf5'
f = h5py.File(file, 'r')

In [4]:
# Get the feature and the label
# The dataset have larger points HHR region [data= Augumentation]
dataset = np.array(f.get('molar_fraction'))
print(dataset.shape)

(50901851, 6)


In [5]:
# Separate the feature and the label
X_old = dataset[:,:-1]
Y = dataset[:,[-1]]
print(X_old.shape,Y.shape)

(50901851, 5) (50901851, 1)


In [14]:
# the product of mass fraction and density to find mole fraction.
# get the mass fraction 
y_nh = X_old[:,0:1]
y_nh2 = X_old[:,1:2]
y_oh = X_old[:,2:3]
# Get the tmperature{The additional input}
temp = X_old[:,4:5]
# get the density
density = X_old[:,3:4]

# A linear relation between mass fractio and mole fraction 
X_nh = y_nh *density
X_nh2 = y_nh2 *density
X_oh = y_oh *density

# The new input 
X = np.c_[X_nh,X_nh2,X_oh,temp]
print(X)

[[3.24108411e-07 7.15551340e-06 1.50022992e-04 1.88831462e+03]
 [3.24264966e-07 7.15909405e-06 1.50018731e-04 1.88830965e+03]
 [3.24387130e-07 7.16189148e-06 1.50015388e-04 1.88830574e+03]
 ...
 [1.03291829e-05 3.26863957e-05 8.84004547e-04 2.29063320e+03]
 [1.01743229e-05 3.21374957e-05 8.88756537e-04 2.29142432e+03]
 [9.85278722e-06 3.10329526e-05 8.92953618e-04 2.29242941e+03]]


In [5]:
# Now normalize the train and test data		
scale_X = MinMaxScaler()
scale_Y = MinMaxScaler()

In [6]:
# Normalize the trainig data
X_n = scale_X.fit_transform(X)
Y_n = scale_Y.fit_transform(Y)

In [7]:
#load the model
mod = 'D:/Education/HRR/Ann reslt/updated_model/updated_model.h5'
model = tf.keras.models.load_model(mod)

In [8]:
# Get summary
model.summary()

Model: "Heat_release_rate"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h_layer1 (Dense)             (None, 40)                200       
_________________________________________________________________
h_layer2 (Dense)             (None, 24)                984       
_________________________________________________________________
h_layer3 (Dense)             (None, 24)                600       
_________________________________________________________________
output_layers (Dense)        (None, 1)                 25        
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Get the data from a randomly selected file.
doc = 'D:/Education/HRR/dino_res_21200.h5'
f1 = h5py.File(doc, 'r')

In [10]:
# Extract the info of chemical marker and the heat release rate
y_nh = np.array(f1['Y_NH'])
y_nh2 = np.array(f1['Y_NH2'])
y_oh = np.array(f1['Y_OH'])
temp = np.array(f1['temper'])
#  The output label
heat = np.array(f1['hr'])

# make the tensor
tensor = np.array([[y_nh],[y_nh2],[y_oh],[temp],[heat]])
tensor = tf.transpose(tensor, perm=[2, 3, 0, 1])
tensor = tf.reshape(tensor, shape=[-1, 5])

X_reconstruct = tensor[:, :-1]
X_reconstruct = np.array(X_reconstruct)
Y_ori = tensor[:, -1]
Y_ori = tf.reshape(Y_ori , shape=[len(Y_ori), 1])
Y_ori =np.array(Y_ori)

In [11]:
# Get the index positions where the NHRR is less than 0.1
NHRR = 0.1
norm_hrr = minmax_scale(Y_ori, axis=0)
mask_1 = tf.where(norm_hrr < NHRR)
redundant_index = mask_1[:, 0]

In [12]:
# Now fit the extracted data to the scale 
X_test_n = scale_X.transform(X_reconstruct)
Y_test_n = scale_Y.transform(Y_ori)
#print(X_test_n, Y_test_n)

In [ ]:
# Predict the result 
pred = model.predict(X_test_n)
print(pred)

In [ ]:
# Inverse the predicted output
pred = scale_Y.inverse_transform(pred)

In [ ]:
# Now map those places where the NHRR <0.1 to 0.[Both in pred and the Y_ori]
Y_ori[redundant_index] = 0
pred[redundant_index] = 0 

In [ ]:
# Plot the fitting of the prediction
plt.scatter(pred, Y_ori, s=1,color='green')
plt.xlabel('y_pred')
plt.ylabel('Y_ori')
# predicted values
plt.plot(Y_ori, Y_ori, color='k')

In [ ]:
#Calculation of the absolute error 
absolute_error = np.abs(pred-Y_ori) 

In [ ]:
# Reshape the predition,Y_ori and Abs_error.
prediction = tf.reshape(pred, shape=[2049, 1024])
Y_ori = tf.reshape(Y_ori, shape=[2049, 1024])
abs_error = tf.reshape(absolute_error, shape=[2049, 1024])

In [ ]:
# save the reconstructed image
c = h5py.File('Recon_Analysis_21200.h5','w')
c.create_dataset('prediction',data=prediction)
c.create_dataset('HRR_original',data=Y_ori)
c.create_dataset('absolute_error',data=abs_error)
c.close()